# Quick Visualizer SPHinXsys - Water/Fluid interactions

## Import required libraries

the VTK and PyVista libararies are not automatically included in jupyter notebook! Make sure you install them in your anaconda prompt, or delete the '#' in the following cell. These libraries only work in Python 3, so make sure your kernel is compatible.

In [1]:
# %pip install pyvista vtk

In [2]:
import glob
import vtk
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
import time

## Import the required files

Make sure you have run the SPHinxSys programm and created the required the .vtp files. Change the directory to what corresponds to your .vtp files (usually ends in 'bin/output/WaterBody____.vtp', but you need to check yourself)

In [3]:
water     = glob.glob("output/WaterBody_*.vtp")
frames    = len(water)
wall      = glob.glob("output/Wall*.vtp")

## Read the required files and generate the grid

Read the required files and make an initial plot, this is just to test whether your code reads the program correctly. More documentation can be found here: https://examples.vtk.org/site/Python/IO/ReadPolyData/ , https://vtk.org/doc/nightly/html/classvtkPoints.html and https://examples.vtk.org/site/Python/Snippets/ReadPolyData/ .

##### 'location' and 'speed' are lists
##### 'grid' and 'velocity' are arrays
Currently the code outputs arrays, but if you prefer lists for other uses, know that you need to call location and speed.

In [4]:
reader   = vtk.vtkXMLPolyDataReader()

In [5]:
def reading(frame):    #This function reads a certain frame and generates the output
    reader.SetFileName(water[frame])
    reader.Update() 
    
    output      = reader.GetOutput()
    points      = output.GetPoints()
    n_points    = output.GetNumberOfPoints()
    speed_array = output.GetPointData().GetArray("Velocity")
    location    = []
    speed       = []
    for i in range(n_points):
        current_point    = points.GetPoint(i)
        current_speed    = speed_array.GetTuple3(i)
        current_speed    = np.linalg.norm(current_speed)
        location.append(current_point)
        speed.append(current_speed)
        
    grid        = np.stack(location)
    velocity    = np.stack(speed   )
    return grid, velocity

In [6]:
def reading_wall(wall):  #This function reads the wall boundary
    reader.SetFileName(wall[0])
    reader.Update() 
    
    output     = reader.GetOutput()
    points     = output.GetPoints()
    n_points   = output.GetNumberOfPoints()
    barrier    = []
    for i in range(n_points):
        barrier.append(points.GetPoint(i))
    grid       = np.stack(barrier)
    return grid

## Make a .gif of your points.

https://docs.pyvista.org/examples/02-plot/gif.html has a great example of how .gif's are created in PyVista. You can change some things according to your preference. Like the particle size, opacity, but you can also adjust the colormap to your preferences. Use https://matplotlib.org/stable/gallery/color/colormap_reference.html where you can see all the colormaps that are included in Matplotlib.

You can change the camera position in many ways, but you can also manually place the camera in a position. Please refer to: https://docs.pyvista.org/api/plotting/_autosummary/pyvista.plotter.camera_position

##### note: 
It is a known problem that the colorbar is not extremely stable in PyVista. As soon as I've found a solution for that I'll implement it


In [7]:
#create the plot itself
plotter = pv.Plotter(notebook=False, off_screen=True)

#This is the wall boundary
#(don't delete the section, it helps the camera position)
plotter.add_points(
    reading_wall(wall),
    opacity= 0.005,                   #set this to zero if you don't want to see the wall at all 
    render_points_as_spheres=True,
    point_size=5                      #change the size of the particles accordingly
)

#These are the initial fluid particles
plotter.add_points(
    reading(0)[0],
    scalars = reading(0)[1],
    opacity= 1,                       #opacity of the water particles. If you want to have a cut through a certain axis, see the next kernel
    cmap='turbo',    
    clim=[0, 0.5],                    
    render_points_as_spheres=True,
    point_size=5                      #change the size of the particles accordingly
)
plotter.set_background("white")
plotter.camera_position = 'xy'        #the first letter is the horizontal axis the second axis is the vertical axis. 

#Open the give and give it a name
plotter.open_gif("test_full_dambreak.gif")   

nframe = 15
print(f"start creating .gif")

starttime = time.time()
midtime   = starttime
for j in range(int(frames)):
    pts, vel  = reading(j)
    plotter.update_coordinates(pts, render=False)
    plotter.update_scalars(vel, render=False)
    plotter.write_frame()
    if j % int(frames/10) == 0:
        print(f"Currently at {j/int(frames/10)*10}% of {int(frames)} frames. Total time {int(time.time() - starttime)} seconds, time since last update {int(time.time() - midtime)} seconds ")
        midtime = time.time()

print(f"All done compiling frames")

plotter.close()
print(f".gif created, enjoy!")

start creating .gif


C:\Users\vikto\anaconda3\Lib\site-packages\pyvista\plotting\plotter.py:4892: PyVistaDeprecationWarning: This method is deprecated and will be removed in a future version of PyVista. Directly modify the points of a mesh in-place instead.
  warnings.warn(
C:\Users\vikto\anaconda3\Lib\site-packages\pyvista\plotting\plotter.py:4814: PyVistaDeprecationWarning: This method is deprecated and will be removed in a future version of PyVista. Directly modify the scalars of a mesh in-place instead.
  warnings.warn(


Currently at 0.0% of 98 frames. Total time 0 seconds, time since last update 0 seconds 
Currently at 10.0% of 98 frames. Total time 4 seconds, time since last update 3 seconds 
Currently at 20.0% of 98 frames. Total time 7 seconds, time since last update 3 seconds 
Currently at 30.0% of 98 frames. Total time 11 seconds, time since last update 3 seconds 
Currently at 40.0% of 98 frames. Total time 15 seconds, time since last update 3 seconds 
Currently at 50.0% of 98 frames. Total time 19 seconds, time since last update 3 seconds 
Currently at 60.0% of 98 frames. Total time 22 seconds, time since last update 3 seconds 
Currently at 70.0% of 98 frames. Total time 26 seconds, time since last update 3 seconds 
Currently at 80.0% of 98 frames. Total time 30 seconds, time since last update 3 seconds 
Currently at 90.0% of 98 frames. Total time 33 seconds, time since last update 3 seconds 
Currently at 100.0% of 98 frames. Total time 37 seconds, time since last update 3 seconds 
All done comp

## Make a .gif of a crossection

This is useful in the case that you want to make a plot of the internal crossection. 

##### note: 
This is a slower method, because you can't use the .update_coordinates command. You have to clear and rewrite each frame. it makes it a bit more slow. But it does give a clear picture of what happens internally


In [8]:
def reading_percentage(frame, direction, cutoff, upperlower):    #This function reads a certain frame and generates the output
    reader.SetFileName(water[frame])
    reader.Update() 
    
    output      = reader.GetOutput()
    points      = output.GetPoints()
    n_points    = output.GetNumberOfPoints()
    speed_array = output.GetPointData().GetArray("Velocity")
    location    = []
    speed       = []
    for i in range(n_points):
        current_point    = points.GetPoint(i)
        current_speed    = speed_array.GetTuple3(i)
        current_speed    = np.linalg.norm(current_speed)
        
        if upperlower == 1 and current_point[direction] > cutoff:
            location.append(current_point)
            speed.append(current_speed)
        elif upperlower == 0 and current_point[direction] < cutoff:
            location.append(current_point)
            speed.append(current_speed)
        
    grid        = np.stack(location)
    velocity    = np.stack(speed   )
    return grid, velocity

In [9]:
#create the plot itself
plotter = pv.Plotter(notebook=False, off_screen=True)

#This is the wall boundary
#(don't delete the section, it helps the camera position)
plotter.add_points(
    reading_wall(wall),
    opacity= 0.0,                   #set this to zero if you don't want to see the wall at all 
    render_points_as_spheres=True,
    point_size=5                      #change the size of the particles accordingly
)

#These are the initial fluid particles
pts, vel  = reading_percentage(0, 1, 0.25, 0)    
plotter.add_points(
        points = pts,
        scalars = vel,
        opacity= 1,                       #opacity of the water particles. If you want to have a cut through a certain axis, see the next kernel
        cmap='turbo',    
        clim=[0, 0.5],                   
        render_points_as_spheres=True,
        point_size=5                      #change this to the particle size of your choosing
)
plotter.set_background("white")
plotter.camera_position = 'xz'           #the first letter is the horizontal axis the second axis is the vertical axis. 


#Open the give and give it a name
plotter.open_gif("test_cross_section.gif")   

nframe = 15
print(f"start creating .gif")

starttime = time.time()
midtime   = starttime
for j in range(int(frames)):
    pts, vel  = reading_percentage(j, 1, 0.25, 0)
    plotter.clear()
    plotter.add_points(
        pts,
        scalars = vel,
        opacity= 1,                       #opacity of the water particles. If you want to have a cut through a certain axis, see the next kernel
        cmap='turbo',    
        clim=[0, 0.5],                    
        render_points_as_spheres=True,
        point_size=5                      #change this to the particle size of your choosing
    )
    
    plotter.write_frame()
    if j % int(frames/10) == 0:
        print(f"Currently at {j/int(frames/10)*10}% of {int(frames)} frames. Total time {int(time.time() - starttime)} seconds, time since last update {int(time.time() - midtime)} seconds ")
        midtime = time.time()
print(f"All done compiling frames")

# Closes and finalizes movie
plotter.close()
print(f".gif created, enjoy!")

start creating .gif
Currently at 0.0% of 98 frames. Total time 0 seconds, time since last update 0 seconds 
Currently at 10.0% of 98 frames. Total time 4 seconds, time since last update 3 seconds 
Currently at 20.0% of 98 frames. Total time 7 seconds, time since last update 3 seconds 
Currently at 30.0% of 98 frames. Total time 11 seconds, time since last update 3 seconds 
Currently at 40.0% of 98 frames. Total time 15 seconds, time since last update 4 seconds 
Currently at 50.0% of 98 frames. Total time 19 seconds, time since last update 3 seconds 
Currently at 60.0% of 98 frames. Total time 23 seconds, time since last update 3 seconds 
Currently at 70.0% of 98 frames. Total time 27 seconds, time since last update 3 seconds 
Currently at 80.0% of 98 frames. Total time 31 seconds, time since last update 4 seconds 
Currently at 90.0% of 98 frames. Total time 34 seconds, time since last update 3 seconds 
Currently at 100.0% of 98 frames. Total time 38 seconds, time since last update 4 se